In [1]:
import pandas as pd
import numpy as np

In [29]:
def AZ_action(df,create_order):
    ##pick the active one information
    active_inf = df[df['need active'] ==1]
    active_inf = active_inf[ active_inf['run_proper'] ==active_inf['run_proper'].max()]
    active_inf = active_inf[active_inf['create order']==active_inf['create order'].min()].iloc[0]
    active_one_order = active_inf['create order']


    ##pick the destory pool
    destory_pool = df[(df['area']==active_inf['area']) & (df['is alive']==1) & (df['create order'] != active_one_order)]

    if len(destory_pool) > 0:
        ##random select once
        rng_num = np.random.randint(low=0, high=(len(destory_pool)))
        selected_destory = destory_pool.iloc[rng_num]
        selected_destory_order = selected_destory['create order']
        copy_active = [1 if selected_destory['card type']==1 else 0]
        copy_runp = [active_inf['run_proper']+1 if selected_destory['card type']==1 else 0]
        copy_eara = [1,2,3]
        copy_eara.remove(active_inf['area'])
        ##destroy it and copy it to df
        df.loc[df['create order'] == selected_destory_order, 'is alive'] = 0
        
        for area in copy_eara:
            if len(df[(df['area']==area) & df['is alive']==1]) <4:
                df = pd.concat([df, pd.DataFrame({
                    "card type":selected_destory['card type'],
                    "area":area,
                    "is alive":1,
                    "need active":copy_active,
                    "create order":create_order,
                    "run_proper":copy_runp},index=[0])])
                create_order = create_order+1

    ##set active to non-active
        df.loc[df['create order'] == active_one_order,'need active'] = 0
        #print(create_order)
    return df,create_order
#display(active_inf)
#display(selected_destory)
#display(df)


In [38]:
record_full = pd.DataFrame()
for trial in range(100):
    #int Gen 1dio + 1AZ + 3AZ
    df = pd.DataFrame()
    create_order=1
    dio  = {"card type":0,"area":2,"is alive":1,"need active":0,"create order":create_order,"run_proper":0}
    create_order = create_order + 1
    #card type 0 dio, card type 1 AZ
    df = pd.concat([df,pd.DataFrame(dio,index=[0])])
    df = pd.concat([df,pd.DataFrame({"card type":1,"area":1,"is alive":1,"need active":0,"create order":create_order,"run_proper":0},index=[0])])
    create_order = create_order + 1
    for int_AZ_gen in range(3):
        df = pd.concat([df,pd.DataFrame({"card type":1,"area":1,"is alive":1,"need active":1,"create order":create_order,"run_proper":0},index=[0])])
        create_order = create_order + 1

    #action start
    while((df['need active'].max() ==1) &(df['create order'].max() <500)):
        df,create_order = AZ_action(df,create_order)

    #record results
    record = {"run":trial+1,
            "creation":create_order-1,
            "number of alive": df['is alive'].sum(),
            "number of dio alive":df[df['card type']==0]['is alive'].sum(),
            "number of AZ alive":df[df['card type']==1]['is alive'].sum(),
            "number of dio generated":df[df['card type']==0]['card type'].count(),
            "number of AZ generated":df[df['card type']==1]['card type'].count(),
            "number of dio alive in area1":df[(df['card type']==0) & (df['area']==1)]['is alive'].sum(),
            "number of dio alive in area2":df[(df['card type']==0) & (df['area']==2)]['is alive'].sum(),
            "number of dio alive in area3":df[(df['card type']==0) & (df['area']==3)]['is alive'].sum(),
            "number of AZ alive in area1":df[(df['card type']==1) & (df['area']==1)]['is alive'].sum(),
            "number of AZ alive in area2":df[(df['card type']==1) & (df['area']==2)]['is alive'].sum(),
            "number of AZ alive in area3":df[(df['card type']==1) & (df['area']==3)]['is alive'].sum(),
            }
    record_full = pd.concat([record_full, pd.DataFrame(record,index=[0])])
record_full

,run,creation,number of alive,number of dio alive,number of AZ alive,number of dio generated,number of AZ generated,number of dio alive in area1,number of dio alive in area2,number of dio alive in area3,number of AZ alive in area1,number of AZ alive in area2,number of AZ alive in area3
0,1,35,11,3,8,10,25,1,1,1,3,3,2
0,2,29,11,3,8,10,19,1,2,0,3,2,3
0,3,500,10,2,8,5,495,2,0,0,2,3,3
0,4,22,11,4,7,10,12,1,0,3,3,3,1
0,5,17,10,4,6,9,8,1,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,96,44,11,3,8,10,34,0,1,2,3,3,2
0,97,19,10,4,6,9,10,0,3,1,3,1,2
0,98,500,10,2,8,5,495,2,0,0,2,3,3
0,99,19,10,4,6,9,10,1,2,1,2,2,2


In [42]:
record_full.describe() #record_full['creation']>=500

,run,creation,number of alive,number of dio alive,number of AZ alive,number of dio generated,number of AZ generated,number of dio alive in area1,number of dio alive in area2,number of dio alive in area3,number of AZ alive in area1,number of AZ alive in area2,number of AZ alive in area3
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000
mean,50.500000,148.760000,10.590000,3.320000,7.270000,8.510000,140.250000,1.320000,1.330000,0.670000,2.38000,2.400000,2.490000
std,29.011492,209.359442,0.494311,1.023758,0.983243,2.124865,211.444545,0.839432,0.910711,0.739301,0.61595,0.550482,0.541136
min,1.000000,17.000000,10.000000,2.000000,6.000000,5.000000,8.000000,0.000000,0.000000,0.000000,1.00000,1.000000,1.000000
25%,25.750000,19.000000,10.000000,2.000000,6.000000,5.000000,10.000000,1.000000,0.000000,0.000000,2.00000,2.000000,2.000000
50%,50.500000,29.000000,11.000000,3.000000,8.000000,10.000000,19.000000,1.500000,2.000000,1.000000,2.00000,2.000000,3.000000
75%,75.250000,500.000000,11.000000,4.000000,8.000000,10.000000,495.000000,2.000000,2.000000,1.000000,3.00000,3.000000,3.000000
max,100.000000,500.000000,11.000000,5.000000,9.000000,10.000000,495.000000,3.000000,3.000000,3.000000,4.00000,3.000000,3.000000


In [11]:
df[(df['is alive']==1 )| (df['need active']==1)]

,card type,area,is alive,need active,create order,run_proper
0,1,1,1,0,4,0
0,1,1,0,1,5,0
0,0,1,1,0,8,0
0,1,3,0,1,12,1
0,1,1,1,0,13,2
0,1,3,0,1,14,2
0,0,1,1,0,17,0
0,1,2,1,0,3616,3602
0,1,3,1,0,3621,3607
0,1,3,1,0,3625,3611


In [8]:
review_df = df[df['is alive']==1]
review_df.groupby(['card type','area']).count()

is alive  need active  create order  run_proper
card type area                                                 
0         1            2            2             2           2
1         1            2            2             2           2
          2            2            2             2           2
          3            3            3             3           3